In [ ]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
t, h1, h2 = sympy.symbols("t h1 h2", real=True)

In [ ]:
phi_0 = sympy.Piecewise(
    ((1 + t) ** 2 * (1 - 2 * t), (t >= -1) & (t <= 0)),
    ((1 - t) ** 2 * (1 + 2 * t), (t >= 0) & (t <= 1)),
    (0, True),
)

phi_1 = sympy.Piecewise(
    ((1 + t) ** 2 * t, (t >= -1) & (t <= 0)),
    ((1 - t) ** 2 * t, (t >= 0) & (t <= 1)),
    (0, True),
)

In [ ]:
choices = [phi_0, phi_1]

In [ ]:
result = {}
for idx, A in enumerate(tqdm(choices)):
    for jdx, B in enumerate(choices):
        for i in range(-1, 2):
            for j in range(-1, 2):
                for kdx, a in enumerate(choices):
                    for ldx, b in enumerate(choices):
                        A_tt = sympy.diff(A, t, t) / h1**2
                        B_tt = sympy.diff(A, t, t) / h2**2
                        A_t = sympy.diff(A, t) / h1
                        B_t = sympy.diff(B, t) / h2

                        a = a.subs(t, t - i)
                        b = b.subs(t, t - j)

                        a_tt = sympy.diff(a, t, t) / h1**2
                        b_tt = sympy.diff(b, t, t) / h2**2
                        a_t = sympy.diff(a, t) / h1
                        b_t = sympy.diff(b, t) / h2

                        INT = (
                            (
                                sympy.integrate(A_tt * a_tt, (t, -1, 1))
                                * sympy.integrate(B * b, (t, -1, 1))
                                + 2
                                * sympy.integrate(A_t * a_t, (t, -1, 1))
                                * sympy.integrate(B_t * b_t, (t, -1, 1))
                                + sympy.integrate(A * a, (t, -1, 1))
                                * sympy.integrate(B_tt * b_tt, (t, -1, 1))
                            )
                            * h1
                            * h2
                        )

                        result[f"{idx}_{jdx}_{kdx}_{ldx}_{i}_{j}"] = sympy.lambdify(
                            (h1, h2), INT
                        )

In [ ]:
for idx, A in enumerate(choices):
    for jdx, B in enumerate(choices):
        sym_int = (
            sympy.integrate(A, (t, -1, 1)) * sympy.integrate(B, (t, -1, 1)) * h1 * h2
        )

        result[f"{idx}_{jdx}"] = sympy.lambdify((h1, h2), sym_int)

In [ ]:
import dill

dill.settings["recurse"] = True
dill.dump(result, open("biharminic_hermite", "wb"))